In [ ]:
from IPython.display import display, Markdown
from typing import List
import pandas as pd
import plotly.express as px
from pydantic import BaseModel, Field
from rl.llm_agent import LLMAgent
from rl.environment import Environment
from rl.code_evaluator import CodeEvaluator
from rl.policies import EpsilonGreedyPolicy
from rl.utils import compute_delta_grade, is_terminate_grade
import json

MAX_EPOCHS = 3

In [ ]:
# List with JSON files name
json_files_coder = [
    "json_files/prompts_clarity_coder.json",
    "json_files/prompts_size_coder.json",
    "json_files/prompts_specificity_coder.json",
    "json_files/prompts_complexity_coder.json"
]

json_files_reviewer = [
    "json_files/prompts_clarity_reviewer.json",
    "json_files/prompts_size_reviewer.json",
    "json_files/prompts_specificity_reviewer.json",
    "json_files/prompts_complexity_reviewer.json"
]

json_files_refiner = [
    "json_files/prompts_prop1_refiner.json",
    "json_files/prompts_prop2_refiner.json",
    "json_files/prompts_prop3_refiner.json",
    "json_files/prompts_prop4_refiner.json"
]

# Extracting all coder prompts into a list
prompts_coder = []

for file_name in json_files_coder:
    # Loading the contents of JSON files
    with open(file_name, "r", encoding="utf-8") as file:
        data = json.load(file)
        for item in data:
            prompts_coder.append(item["prompt"])

# Extracting all reviewer properties into a list
reviewer_properties = []       

for file_name in json_files_reviewer:
    # Loading the contents of JSON files
    with open(file_name, "r", encoding="utf-8") as file:
        data = json.load(file)
        for item in data:
            reviewer_properties.append(item["propriedade"])

# Extracting all refiner properties into a list
refiner_properties = []     

for file_name in json_files_coder:
    # Loading the contents of JSON files
    with open(file_name, "r", encoding="utf-8") as file:
        data = json.load(file)
        for item in data:
            refiner_properties.append(item["propriedade"])             

In [ ]:
#TODO: Make the main loop with this
for prompt_coder in prompts_coder:
    for reviewer_property in reviewer_properties:
        for refiner_property in refiner_properties:
            for i in range(1,6):
                print(i)   